In [ ]:
# Import base python libraries
import uuid
import json
import os

## Method 1: Use OpenDSScmd Command Line
Preferred. OpenDSScmd is updated with latest CIM GMDM profile and IEEE 1547 settings

In [ ]:
# Confirm that opendsscmd is running
!which opendsscmd

In [ ]:
# Specify OpenDSS master-file
dss_file = "./dss/IEEE13Node.dss"
sub = "IEEE13"
geo = "demo"
subgeo = "small"
name = "ieee13training"
feeder_mrid = str(uuid.uuid4())
destination = "IEEE13Demo.xml"

In [ ]:
with open('cim_converter.dss', 'a') as df:
    df.write(f'redirect {dss_file} \n')
    #df.write('uuids file=base/{:s}_uuids.dat'.format (root))
    df.write('set maxiterations=20 \n')
    df.write('set tolerance=1e-5 \n')
    df.write('solve \n')
    df.write(f'export cim100 fid={feeder_mrid} substation={sub} geo={geo} subgeo={subgeo} file={destination}')

In [ ]:
# Run command line exporter:
!opendsscmd cim_converter.dss

## Method 2: Use OpenDSS Direct
(Unknown which profile version opendss direct is using)

In [ ]:
# Import OpenDSS Direct
import opendssdirect as dss

In [ ]:
# Load IEEE 13 Node OpenDSS Sample Case
dss.run_command('Redirect ./dss/IEEE13Node.dss')
dss.run_command('Compile ./dss/IEEE13Node.dss')
dss.Solution.Solve()

In [ ]:
# Convert to CIM 100 XML
dss.run_command(f'export cim100 fid={feeder_mrid} substation=ieee13 subgeo=small geo=sample file=ieee13demo1.xml')
os.chdir('..') #return to main directory

## Upload to Database

In [ ]:
# Load model into Blazegraph Database
!curl -s -D- -H 'Content-Type: application/xml' --upload-file 'IEEE13Demo.xml' -X POST 'http://localhost:8889/bigdata/namespace/kb/sparql'

## Optional: Use CIMHub to add SCADA points, houses, and DER

In [ ]:
# Use CIMHub Library to print all feeders in database
!python3 ../CIMHub/src_python/cimhub/ListFeeders.py

In [ ]:
# Create SCADA points for all objects in the feeder
!python3 ../CIMHub/src_python/cimhub/ListMeasureables.py ../CIMHub/cimhubconfig.json {name} {"_"+feeder_mrid}
!for f in `ls -1 *txt`; do python3 ../CIMHub/src_python/cimhub/InsertMeasurements.py ../CIMHub/cimhubconfig.json $f uuidfile.json; done